## 1. Import Libraries and Setup

# Fine-tune Stable Diffusion on Custom Dataset

This notebook trains (fine-tunes) a Stable Diffusion model on your COCO + Flickr image-caption dataset to generate custom images specific to your domain.

**Training Options:**
- Full fine-tuning (all parameters)
- LoRA fine-tuning (parameter-efficient, recommended for limited GPU)
- Text encoder fine-tuning
- UNet fine-tuning

**Hardware Support:**
- Works on CPU (slow, for debugging)
- Single GPU (recommended minimum: 8GB VRAM)
- Multi-GPU with DistributedDataParallel
- HiperGator multi-node setup

# Fine-tune Stable Diffusion on Custom Dataset

This notebook trains (fine-tunes) a Stable Diffusion model on your COCO + Flickr image-caption dataset to generate custom images specific to your domain.

**Training Options:**
- Full fine-tuning (all parameters)
- LoRA fine-tuning (parameter-efficient, recommended for limited GPU)
- Text encoder fine-tuning
- UNet fine-tuning

**Hardware Support:**
- Works on CPU (slow, for debugging)
- Single GPU (recommended minimum: 8GB VRAM)
- Multi-GPU with DistributedDataParallel
- HiperGator multi-node setup

## 1. Import Libraries and Setup

In [1]:
import os
import sys
import json
import math
import random
from pathlib import Path
from datetime import datetime

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW
from torch.cuda.amp import autocast, GradScaler
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP

# Hugging Face libraries
from transformers import CLIPTextModel, CLIPTokenizer, AutoTokenizer, BertTokenizer, BertModel
from diffusers import AutoencoderKL, UNet2DConditionModel, DDPMScheduler, StableDiffusionPipeline, LDMTextToImagePipeline
from diffusers.optimization import get_cosine_schedule_with_warmup

# Image processing
import torchvision.transforms as T
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Setup
def set_seed(seed=42):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)
print("✓ Libraries imported successfully")


c:\Users\Aishu\anaconda3\envs\eoi\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✓ Libraries imported successfully


## 2. Configuration and Hardware Setup

In [7]:
# ============ CONFIGURATION ============
class Config:
    # Model Configuration
    # Choose one:
    # - "runwayml/stable-diffusion-v1-5"        # 860M params, 20-40 hrs/epoch, best quality
    # - "stabilityai/stable-diffusion-2-base"   # 500M params, 10-20 hrs/epoch, good balance
    # - "stabilityai/sd-turbo"                  # 160M params, 2-4 hrs/epoch, RECOMMENDED for fast
    model_id = "stabilityai/sd-turbo"
    image_size = 512  # SD uses 512x512 (can reduce to 256 for faster training)
    
    # Training Configuration
    num_epochs = 10
    train_batch_size = 2  # Reduced for stability (increase if you have >8GB VRAM)
    gradient_accumulation_steps = 4  # Simulates batch_size=8
    learning_rate = 1e-4
    max_grad_norm = 1.0
    weight_decay = 0.01
    
    # Warmup and scheduling
    lr_warmup_steps = 500
    num_train_timesteps = 1000
    
    # Fine-tuning options
    fine_tune_text_encoder = False  # Set to True only if you have >12GB VRAM
    use_lora = False  # Set True for parameter-efficient LoRA fine-tuning
    lora_rank = 16  # LoRA matrix rank
    
    # Mixed precision
    use_mixed_precision = torch.cuda.is_available()
    
    # Checkpointing
    checkpoints_dir = "../models/checkpoints"
    save_interval = 2  # Save checkpoint every N epochs
    
    # Validation
    num_validation_images = 4
    validation_interval = 2
    validation_prompts = [
        "a photo of a cat",
        "a beautiful landscape with mountains",
        "a person smiling at the camera",
        "a delicious pizza on a plate"
    ]

# Create config instance
config = Config()

# Hardware setup
NUM_GPUS = torch.cuda.device_count()
USE_GPU = torch.cuda.is_available()
DEVICE = torch.device('cuda' if USE_GPU else 'cpu')
LOCAL_RANK = int(os.environ.get('LOCAL_RANK', 0))
WORLD_SIZE = int(os.environ.get('WORLD_SIZE', 1))

print(f"Device: {DEVICE}")
print(f"Number of GPUs: {NUM_GPUS}")
if USE_GPU:
    for i in range(NUM_GPUS):
        print(f"  GPU {i}: {torch.cuda.get_device_name(i)}")

# Adjust batch size for multi-GPU
if NUM_GPUS > 1:
    config.train_batch_size = max(1, config.train_batch_size // NUM_GPUS)

print(f"\nTraining Configuration:")
print(f"  Model ID: {config.model_id}")
print(f"  Image Size: {config.image_size}x{config.image_size}")
print(f"  Batch Size: {config.train_batch_size}")
print(f"  Effective Batch Size (with accumulation): {config.train_batch_size * config.gradient_accumulation_steps}")
print(f"  Epochs: {config.num_epochs}")
print(f"  Learning Rate: {config.learning_rate}")
print(f"  Fine-tune Text Encoder: {config.fine_tune_text_encoder}")
print(f"  Use LoRA: {config.use_lora}")
print(f"  Mixed Precision: {config.use_mixed_precision}")

# Create checkpoint directory
os.makedirs(config.checkpoints_dir, exist_ok=True)

Device: cpu
Number of GPUs: 0

Training Configuration:
  Model ID: stabilityai/sd-turbo
  Image Size: 512x512
  Batch Size: 2
  Effective Batch Size (with accumulation): 8
  Epochs: 10
  Learning Rate: 0.0001
  Fine-tune Text Encoder: False
  Use LoRA: False
  Mixed Precision: False


## 3. Load Custom Dataset

In [3]:
# Setup path to access custom modules
root_directory = os.path.dirname(os.getcwd())
sys.path.append(os.path.join(root_directory, 'src'))

from dataloaders_text import caption_dataset

# Initialize dataloader handler
print("Loading dataset using caption_dataset.get_dataloader()...")
dataloader_handler = caption_dataset()

# Get dataloaders directly - much simpler!
train_dataloader, train_sampler = dataloader_handler.get_dataloader(
    partition="train", 
    batch_size=config.train_batch_size,
    distributed=False
)

val_dataloader, val_sampler = dataloader_handler.get_dataloader(
    partition="val", 
    batch_size=config.train_batch_size,
    distributed=False
)

print(f"\n✓ Dataloaders loaded successfully!")
print(f"  Train batches per epoch: {len(train_dataloader)}")
print(f"  Val batches per epoch: {len(val_dataloader)}")

# Test a batch to see data format
print("\nTesting data format from dataloader...")
sample_batch = next(iter(train_dataloader))
print(f"  Image shape: {sample_batch[0].shape}")  # Should be [B, C, H, W]
print(f"  Text: {sample_batch[1]}") 


Loading dataset using caption_dataset.get_dataloader()...
  unique_image_identifier                                            caption  \
0          1000092795.jpg   Two young guys with shaggy hair look at their...   
1          1000092795.jpg   Two young , White males are outside near many...   
2          1000092795.jpg   Two men in green shirts are standing in a yard .   
3          1000092795.jpg       A man in a blue shirt standing in a garden .   
4          1000092795.jpg            Two friends enjoy time spent together .   

                                          source_dir  source  
0  c:\Users\Aishu\Downloads\AML-2\Echoes-of-Imagi...  Flickr  
1  c:\Users\Aishu\Downloads\AML-2\Echoes-of-Imagi...  Flickr  
2  c:\Users\Aishu\Downloads\AML-2\Echoes-of-Imagi...  Flickr  
3  c:\Users\Aishu\Downloads\AML-2\Echoes-of-Imagi...  Flickr  
4  c:\Users\Aishu\Downloads\AML-2\Echoes-of-Imagi...  Flickr  


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


  unique_image_identifier                                            caption  \
0          1000092795.jpg   Two young guys with shaggy hair look at their...   
1          1000092795.jpg   Two young , White males are outside near many...   
2          1000092795.jpg   Two men in green shirts are standing in a yard .   
3          1000092795.jpg       A man in a blue shirt standing in a garden .   
4          1000092795.jpg            Two friends enjoy time spent together .   

                                          source_dir  source  
0  c:\Users\Aishu\Downloads\AML-2\Echoes-of-Imagi...  Flickr  
1  c:\Users\Aishu\Downloads\AML-2\Echoes-of-Imagi...  Flickr  
2  c:\Users\Aishu\Downloads\AML-2\Echoes-of-Imagi...  Flickr  
3  c:\Users\Aishu\Downloads\AML-2\Echoes-of-Imagi...  Flickr  
4  c:\Users\Aishu\Downloads\AML-2\Echoes-of-Imagi...  Flickr  

✓ Dataloaders loaded successfully!
  Train batches per epoch: 119609
  Val batches per epoch: 14951

Testing data format from dataloader...


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


  Image shape: torch.Size([4, 3, 64, 64])
  Text: ('Horses in fenced area with grass and hay and adults nearby.', ' three people are dressed up as clowns in a city street .', ' A woman with a camera as many people behind her walk across the water on a walkway .', 'Two women walking on a side walk next to a street containing a car and a big white truck. ')


## 4. Load Pretrained Model Components

In [8]:
print("\n" + "="*60)
print("DIAGNOSTIC: Checking Model Download and Cache")
print("="*60)

import urllib.request
import socket

# Check internet connection
print("\n1. Checking internet connection...")
try:
    urllib.request.urlopen('https://huggingface.co', timeout=5)
    print("   ✓ Internet connection OK")
except:
    print("   ⚠️ WARNING: No internet connection to HuggingFace Hub!")
    print("     Models may already be cached locally")

# Check HuggingFace cache
hf_cache = Path.home() / ".cache" / "huggingface" / "hub"
print(f"\n2. HuggingFace cache location: {hf_cache}")
print(f"   Cache exists: {hf_cache.exists()}")

if hf_cache.exists():
    cached_models = list(hf_cache.glob("*"))
    print(f"   Cached items: {len(cached_models)}")
    if cached_models:
        print(f"   Examples: {[m.name[:50] for m in cached_models[:3]]}")

# Check if model is already cached
model_name = config.model_id.replace("/", "--")
cached_model_path = hf_cache / f"models--{model_name}"
print(f"\n3. Checking for cached model: {config.model_id}")
print(f"   Path: {cached_model_path}")
print(f"   Cached: {cached_model_path.exists()}")

# Show alternatives if needed
if not cached_model_path.exists():
    print("\n⚠️ Model not cached. Download will occur during model loading.")
    print("   Make sure you have:")
    print("   - Stable internet connection")
    print("   - ~5GB free disk space for model + cache")
    print("   - Read/write permissions in ~/.cache/")

print("\n" + "="*60)


DIAGNOSTIC: Checking Model Download and Cache

1. Checking internet connection...
   ✓ Internet connection OK

2. HuggingFace cache location: C:\Users\Aishu\.cache\huggingface\hub
   Cache exists: True
   Cached items: 12
   Examples: ['.locks', 'datasets--ag_news', 'datasets--imdb']

3. Checking for cached model: stabilityai/sd-turbo
   Path: C:\Users\Aishu\.cache\huggingface\hub\models--stabilityai--sd-turbo
   Cached: False

⚠️ Model not cached. Download will occur during model loading.
   Make sure you have:
   - Stable internet connection
   - ~5GB free disk space for model + cache
   - Read/write permissions in ~/.cache/



In [9]:
print(f"\nLoading pretrained model: {config.model_id}")

# All these models use standard Stable Diffusion architecture with CLIP tokenizer
print("✓ Loading Stable Diffusion model")

try:
    # Load tokenizer (CLIP tokenizer - standard for all SD models)
    tokenizer = CLIPTokenizer.from_pretrained(config.model_id, subfolder="tokenizer")
    
    # Load text encoder (CLIP text model - standard for all SD models)
    text_encoder = CLIPTextModel.from_pretrained(config.model_id, subfolder="text_encoder")
    
    # Load VAE (for encoding images to latents)
    vae = AutoencoderKL.from_pretrained(config.model_id, subfolder="vae")
    
    # Load UNet for diffusion
    unet = UNet2DConditionModel.from_pretrained(config.model_id, subfolder="unet")
    
    # Load noise scheduler
    noise_scheduler = DDPMScheduler.from_pretrained(config.model_id, subfolder="scheduler")
    
    print("  ✓ All components loaded successfully")
    
except Exception as e:
    print(f"  ✗ Error loading model: {e}")
    print("\n  Trying fallback: using runwayml/stable-diffusion-v1-5")
    
    tokenizer = CLIPTokenizer.from_pretrained("runwayml/stable-diffusion-v1-5", subfolder="tokenizer")
    text_encoder = CLIPTextModel.from_pretrained("runwayml/stable-diffusion-v1-5", subfolder="text_encoder")
    vae = AutoencoderKL.from_pretrained("runwayml/stable-diffusion-v1-5", subfolder="vae")
    unet = UNet2DConditionModel.from_pretrained("runwayml/stable-diffusion-v1-5", subfolder="unet")
    noise_scheduler = DDPMScheduler.from_pretrained("runwayml/stable-diffusion-v1-5", subfolder="scheduler")
    
    print("  ✓ Fallback model loaded successfully")

# Move to device
text_encoder = text_encoder.to(DEVICE)
vae = vae.to(DEVICE)
unet = unet.to(DEVICE)

# Set to eval mode (we'll only fine-tune specific components)
vae.eval()
text_encoder.eval() if not config.fine_tune_text_encoder else text_encoder.train()
unet.train()

print("\n✓ Model components loaded and ready")
print(f"  Text Encoder: {text_encoder.__class__.__name__}")
print(f"  VAE: {vae.__class__.__name__}")
print(f"  UNet: {unet.__class__.__name__}")
print(f"  Tokenizer: {tokenizer.__class__.__name__}")
print(f"  Device: {DEVICE}")


Loading pretrained model: stabilityai/sd-turbo
✓ Loading Stable Diffusion model


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not in

  ✓ All components loaded successfully

✓ Model components loaded and ready
  Text Encoder: CLIPTextModel
  VAE: AutoencoderKL
  UNet: UNet2DConditionModel
  Tokenizer: CLIPTokenizer
  Device: cpu


## 5. Prepare Dataset and DataLoader

In [10]:
# Note: Dataloaders are already prepared above!
# The dataloader returns: (image, caption) where caption is a string

# For fine-tuning with diffusion, we need to:
# 1. Encode images to latent space
# 2. Tokenize captions and get text embeddings with proper shape [batch, seq_length, embedding_dim]

# Create a wrapper to pre-encode images and tokenize captions
class FineTuningWrapper:
    def __init__(self, dataloader, vae, tokenizer, text_encoder, device, config):
        """
        Wraps dataloader to pre-encode images to latents and tokenize captions.
        """
        self.dataloader = dataloader
        self.vae = vae
        self.tokenizer = tokenizer
        self.text_encoder = text_encoder
        self.device = device
        self.config = config
        
    def __iter__(self):
        """Pre-encode each batch's images to latents and tokenize captions."""
        for batch_idx, (images, captions) in enumerate(self.dataloader):
            images = images.to(self.device)
            
            # Pre-encode images to latent space
            with torch.no_grad():
                # Resize if needed (dataloader uses 64x64, we might need 512x512)
                if images.shape[-1] != self.config.image_size:
                    images = F.interpolate(
                        images, 
                        size=(self.config.image_size, self.config.image_size), 
                        mode='bilinear',
                        align_corners=False
                    )
                
                # Encode to latents
                latents = self.vae.encode(images).latent_dist.sample()
                latents = latents * 0.18215  # Stable Diffusion scaling
                
                # Tokenize captions (convert to list if necessary)
                if isinstance(captions, torch.Tensor):
                    # If captions are already tokens, convert to input_ids format
                    input_ids = captions.to(self.device)
                else:
                    # If captions are strings, tokenize them
                    if not isinstance(captions, (list, tuple)):
                        captions = [captions]  # Handle single caption
                    
                    tokens = self.tokenizer(
                        list(captions),
                        padding="max_length",
                        max_length=self.tokenizer.model_max_length,
                        truncation=True,
                        return_tensors="pt"
                    )
                    input_ids = tokens.input_ids.to(self.device)
                
                # Get text embeddings with proper shape [batch, seq_length, embedding_dim]
                encoder_hidden_states = self.text_encoder(input_ids)[0]  # Shape: [batch, seq_length, 768]
            
            yield {
                "latent": latents,
                "encoder_hidden_states": encoder_hidden_states
            }
    
    def __len__(self):
        return len(self.dataloader)


# Wrap the dataloader
print("\nWrapping dataloader for fine-tuning...")
wrapped_train_dataloader = FineTuningWrapper(
    train_dataloader, 
    vae, 
    tokenizer, 
    text_encoder,
    DEVICE,
    config
)

print(f"✓ Dataloader wrapped successfully")
print(f"  Total batches: {len(wrapped_train_dataloader)}")



Wrapping dataloader for fine-tuning...
✓ Dataloader wrapped successfully
  Total batches: 119609


## 6. Setup Training: Optimizer and Scheduler

In [11]:
# Setup optimizer - only optimize trainable parameters
trainable_params = list(unet.parameters())
if config.fine_tune_text_encoder:
    trainable_params += list(text_encoder.parameters())

optimizer = AdamW(
    trainable_params,
    lr=config.learning_rate,
    weight_decay=config.weight_decay,
    eps=1e-08
)

# Calculate total training steps
num_update_steps_per_epoch = math.ceil(
    len(train_dataloader) / config.gradient_accumulation_steps
)
max_train_steps = config.num_epochs * num_update_steps_per_epoch

# Learning rate scheduler
lr_scheduler = get_cosine_schedule_with_warmup(
    optimizer,
    num_warmup_steps=config.lr_warmup_steps,
    num_training_steps=max_train_steps
)

# Mixed precision training
scaler = GradScaler() if config.use_mixed_precision else None

print(f"\n✓ Optimizer and scheduler configured")
print(f"  Trainable parameters: {sum(p.numel() for p in trainable_params if p.requires_grad):,}")
print(f"  Total training steps: {max_train_steps}")
print(f"  Learning rate: {config.learning_rate}")
print(f"  Mixed precision: {config.use_mixed_precision}")


✓ Optimizer and scheduler configured
  Trainable parameters: 865,910,724
  Total training steps: 299030
  Learning rate: 0.0001
  Mixed precision: False


## 7. Training Loop

In [ ]:
def train_epoch(epoch_num, wrapped_dataloader, optimizer, lr_scheduler, scaler):
    """Train for one epoch."""
    progress_bar = tqdm(total=len(wrapped_dataloader), desc=f"Epoch {epoch_num}")
    losses = []
    
    for step, batch in enumerate(wrapped_dataloader):
        # Get batch data
        latents = batch["latent"]  # Already on device, shape: [B, 4, H, W]
        encoder_hidden_states = batch["encoder_hidden_states"]  # Already on device, shape: [B, seq_len, 768]
        
        # Sample random timesteps
        timesteps = torch.randint(
            0, config.num_train_timesteps, (latents.shape[0],), device=DEVICE
        ).long()
        
        # Sample noise to add to latents
        noise = torch.randn_like(latents)
        
        # Add noise to latents (forward diffusion process)
        noisy_latents = noise_scheduler.add_noise(latents, noise, timesteps)
        
        # Predict noise residual with text conditioning
        if config.use_mixed_precision:
            with autocast():
                model_pred = unet(
                    noisy_latents,
                    timesteps,
                    encoder_hidden_states
                ).sample
                
                # Calculate loss (L2 loss between predicted and actual noise)
                loss = F.mse_loss(model_pred, noise, reduction="mean")
        else:
            model_pred = unet(
                noisy_latents,
                timesteps,
                encoder_hidden_states
            ).sample
            
            # Calculate loss
            loss = F.mse_loss(model_pred, noise, reduction="mean")
        
        # Backward pass with gradient accumulation
        if config.use_mixed_precision:
            scaler.scale(loss).backward()
        else:
            loss.backward()
        
        losses.append(loss.detach().item())
        
        # Update weights
        if (step + 1) % config.gradient_accumulation_steps == 0:
            if config.use_mixed_precision:
                scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(trainable_params, config.max_grad_norm)
            
            if config.use_mixed_precision:
                scaler.step(optimizer)
                scaler.update()
            else:
                optimizer.step()
            
            lr_scheduler.step()
            optimizer.zero_grad()
        
        progress_bar.update(1)
        progress_bar.set_postfix({"loss": f"{loss.item():.4f}"})
    
    progress_bar.close()
    avg_loss = np.mean(losses)
    return avg_loss


# Training loop
print("\n" + "="*60)
print("Starting Fine-tuning...")
print("="*60)

history = {"train_loss": [], "epoch": []}

try:
    for epoch in range(config.num_epochs):
        print(f"\nEpoch {epoch+1}/{config.num_epochs}")
        
        # Train
        avg_loss = train_epoch(epoch+1, wrapped_train_dataloader, optimizer, lr_scheduler, scaler)
        history["train_loss"].append(avg_loss)
        history["epoch"].append(epoch+1)
        
        print(f"Average loss: {avg_loss:.4f}")
        
        # Save checkpoint
        if (epoch + 1) % config.save_interval == 0:
            checkpoint_path = os.path.join(
                config.checkpoints_dir,
                f"checkpoint-epoch-{epoch+1}"
            )
            os.makedirs(checkpoint_path, exist_ok=True)
            
            unet.save_pretrained(os.path.join(checkpoint_path, "unet"))
            if config.fine_tune_text_encoder:
                text_encoder.save_pretrained(os.path.join(checkpoint_path, "text_encoder"))
            
            print(f"✓ Checkpoint saved: {checkpoint_path}")

except KeyboardInterrupt:
    print("\nTraining interrupted by user")
except Exception as e:
    print(f"\nError during training: {e}")
    import traceback
    traceback.print_exc()

print("\n✓ Training complete!")



Starting Fine-tuning...

Epoch 1/10


Epoch 1:   0%|          | 0/119609 [00:00<?, ?it/s]

Epoch 1:   0%|          | 2/119609 [05:06<4856:50:48, 146.18s/it, loss=0.1102]

## 8. Plot Training Loss

In [ ]:
# Plot training loss
if history["train_loss"]:
    plt.figure(figsize=(10, 6))
    plt.plot(history["epoch"], history["train_loss"], marker='o', linewidth=2)
    plt.xlabel("Epoch", fontsize=12)
    plt.ylabel("Average Loss", fontsize=12)
    plt.title("Training Loss Over Epochs", fontsize=14, fontweight='bold')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig("../results/training_loss.png", dpi=150)
    plt.show()
    
    print(f"✓ Training loss plot saved")
else:
    print("No training history available")

## 9. Load Fine-tuned Model for Inference

In [ ]:
# Create pipeline with fine-tuned model
checkpoint_dir = os.path.join(config.checkpoints_dir, f"checkpoint-epoch-{config.num_epochs}")

if os.path.exists(checkpoint_dir):
    print(f"Loading fine-tuned model from: {checkpoint_dir}")
    
    # Load fine-tuned components
    unet_finetuned = UNet2DConditionModel.from_pretrained(
        os.path.join(checkpoint_dir, "unet")
    )
    
    text_encoder_finetuned = CLIPTextModel.from_pretrained(
        os.path.join(checkpoint_dir, "text_encoder")
    ) if config.fine_tune_text_encoder else CLIPTextModel.from_pretrained(
        config.model_id, subfolder="text_encoder"
    )
    
    # Create pipeline with fine-tuned components
    pipe_finetuned = StableDiffusionPipeline.from_pretrained(
        config.model_id,
        unet=unet_finetuned,
        text_encoder=text_encoder_finetuned,
        torch_dtype=torch.float32
    ).to(DEVICE)
    
    print("✓ Fine-tuned pipeline created successfully")
else:
    print(f"Checkpoint directory not found: {checkpoint_dir}")
    print("Using original pretrained model for inference")
    
    pipe_finetuned = StableDiffusionPipeline.from_pretrained(
        config.model_id,
        torch_dtype=torch.float32
    ).to(DEVICE)

## 10. Test Fine-tuned Model with Inference

In [ ]:
# Generate test images with different prompts
test_prompts = [
    "a dog playing in the park",
    "a woman reading a book in a cafe",
    "a sunset over mountains",
    "children building a sandcastle at the beach",
    "a cat sitting on a windowsill"
]

print(f"Generating test images with fine-tuned model on device: {DEVICE}\n")
print("=" * 60)

generated_images = []

for prompt in test_prompts:
    print(f"\nPrompt: {prompt}")
    
    with torch.no_grad():
        images = pipe_finetuned(
            prompt,
            num_inference_steps=50,
            guidance_scale=7.5,
            height=512,
            width=512,
            generator=torch.Generator(device=DEVICE).manual_seed(42)
        ).images
    
    generated_images.append(images[0])
    print(f"✓ Generated 512x512 image successfully")

print("\n" + "=" * 60)
print(f"Generated {len(generated_images)} test images")
print("\nSample outputs saved (showing first 3):")

# Display first 3 images
fig, axes = plt.subplots(1, min(3, len(generated_images)), figsize=(15, 5))
if len(generated_images) == 1:
    axes = [axes]
    
for idx, (ax, img) in enumerate(zip(axes, generated_images[:3])):
    ax.imshow(img)
    ax.set_title(test_prompts[idx][:30] + "...", fontsize=10)
    ax.axis('off')
    
plt.tight_layout()
plt.savefig(os.path.join(config.results_dir, "test_inference.png"), dpi=150, bbox_inches='tight')
plt.show()

print("\nTest images saved to: " + os.path.join(config.results_dir, "test_inference.png"))

## 11. Multi-GPU Training and HiperGator Deployment

### Running on Multi-GPU Locally:
```bash
# Single node, 2 GPUs
torchrun --nproc_per_node=2 fine_tune_distributed.py
```

### HiperGator SBATCH Configuration:
```bash
#!/bin/bash
#SBATCH --nodes=2
#SBATCH --ntasks-per-node=1
#SBATCH --gpus-per-node=a100:2
#SBATCH --cpus-per-task=16
#SBATCH --mem=96GB
#SBATCH --time=12:00:00
#SBATCH --job-name=sd-finetune

module load python cuda/11.8
source venv/bin/activate

torchrun --nproc_per_node=2 --nnodes=2 \
         --master_addr=$MASTER_ADDR --master_port=29500 \
         fine_tune_distributed.py
```

### Key Considerations:
- **Memory**: A100 GPUs (40GB) can handle batch_size=8-16
- **Mixed Precision**: Enabled by default (reduces memory ~2x)
- **Gradient Accumulation**: Simulates larger batches
- **Data Loading**: Pre-encoded latents reduce I/O overhead
- **Checkpointing**: Save every N epochs to resume if interrupted

In [ ]:
# Generate images using fine-tuned model
test_prompts = [
    "a photo of a cat sleeping",
    "a beautiful sunset over mountains",
    "a person laughing at camera",
    "delicious food on a plate"
]

print("Generating images with fine-tuned model...\n")

fig, axes = plt.subplots(2, 2, figsize=(12, 12))
axes = axes.flatten()

for idx, prompt in enumerate(test_prompts):
    print(f"Generating: '{prompt}'")
    
    with torch.no_grad():
        image = pipe_finetuned(
            prompt,
            num_inference_steps=30,
            guidance_scale=7.5,
            height=512,
            width=512,
            num_images_per_prompt=1
        ).images[0]
    
    axes[idx].imshow(image)
    axes[idx].set_title(f"'{prompt}'", fontsize=10, wrap=True)
    axes[idx].axis('off')

plt.tight_layout()
plt.savefig("../results/finetuned_generation_samples.png", dpi=150, bbox_inches='tight')
plt.show()

print("\n✓ Sample generation complete!")

## 12. LoRA Fine-tuning (Parameter-Efficient Alternative)

LoRA (Low-Rank Adaptation) fine-tunes only small matrices added to the model, reducing:
- **Memory**: ~80% reduction
- **Storage**: Checkpoint size ~50MB instead of 4GB
- **Time**: Faster convergence

Uncomment the code below to use LoRA instead of full fine-tuning.

In [ ]:
# ================================
# 11. CLIP Alignment Evaluation
# ================================
print("\n" + "="*70)
print("CLIP ALIGNMENT EVALUATION - Pretrained vs Fine-tuned")
print("="*70)

try:
    # Install clip if needed
    import subprocess
    import sys
    try:
        import clip
    except ImportError:
        print("Installing CLIP...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "openai-clip", "-q"])
        import clip
    
    from PIL import Image as PILImage
    import torch.nn.functional as F_nn
    
    # Load CLIP model
    device_clip = "cuda" if torch.cuda.is_available() else "cpu"
    clip_model, clip_preprocess = clip.load("ViT-B/32", device=device_clip)
    clip_model.eval()
    
    print(f"✓ CLIP model loaded on {device_clip}")
    
    def calculate_clip_score(image, text, model, preprocess, device):
        """Calculate CLIP alignment score between image and text (0-100)"""
        try:
            # Preprocess image
            if isinstance(image, PILImage.Image):
                image_input = preprocess(image).unsqueeze(0).to(device)
            else:
                image_input = image.unsqueeze(0).to(device)
            
            # Tokenize text
            text_input = clip.tokenize([text]).to(device)
            
            # Get embeddings
            with torch.no_grad():
                image_features = model.encode_image(image_input)
                text_features = model.encode_text(text_input)
                
                # Normalize
                image_features = image_features / image_features.norm(dim=-1, keepdim=True)
                text_features = text_features / text_features.norm(dim=-1, keepdim=True)
                
                # Calculate cosine similarity
                similarity = (image_features @ text_features.t()).squeeze()
                score = float(similarity.cpu().numpy()) * 100  # Scale to 0-100
            
            return score
        except Exception as e:
            print(f"Error calculating CLIP score: {e}")
            return None
    
    # Evaluate on test prompts
    print("\n" + "-"*70)
    print("Evaluating on test prompts (Pretrained vs Fine-tuned)")
    print("-"*70)
    
    pretrained_scores = []
    finetuned_scores = []
    
    for idx, prompt in enumerate(test_prompts):
        print(f"\n📊 Prompt {idx + 1}:")
        print(f"  Text: {prompt}")
        
        try:
            # Generate with PRETRAINED model
            with torch.no_grad():
                generated_pretrained = pipe_pretrained(
                    prompt=prompt,
                    num_inference_steps=50,
                    guidance_scale=7.5,
                    generator=torch.Generator(device=device).manual_seed(42)
                ).images[0]
            
            # Generate with FINE-TUNED model
            with torch.no_grad():
                generated_finetuned = pipe(
                    prompt=prompt,
                    num_inference_steps=50,
                    guidance_scale=7.5,
                    generator=torch.Generator(device=device).manual_seed(42)
                ).images[0]
            
            # Calculate CLIP scores
            score_pretrained = calculate_clip_score(generated_pretrained, prompt, clip_model, clip_preprocess, device_clip)
            score_finetuned = calculate_clip_score(generated_finetuned, prompt, clip_model, clip_preprocess, device_clip)
            
            if score_pretrained and score_finetuned:
                pretrained_scores.append(score_pretrained)
                finetuned_scores.append(score_finetuned)
                
                improvement = score_finetuned - score_pretrained
                improvement_pct = (improvement / score_pretrained * 100) if score_pretrained > 0 else 0
                
                print(f"  Pretrained CLIP Score: {score_pretrained:.2f}/100")
                print(f"  Fine-tuned CLIP Score: {score_finetuned:.2f}/100")
                print(f"  Improvement: {improvement:+.2f} ({improvement_pct:+.1f}%)")
                
        except Exception as e:
            print(f"  ✗ Error: {e}")
            continue
    
    # Summary statistics
    if pretrained_scores and finetuned_scores:
        print("\n" + "-"*70)
        print("CLIP ALIGNMENT SUMMARY")
        print("-"*70)
        
        avg_pretrained = np.mean(pretrained_scores)
        avg_finetuned = np.mean(finetuned_scores)
        avg_improvement = avg_finetuned - avg_pretrained
        avg_improvement_pct = (avg_improvement / avg_pretrained * 100) if avg_pretrained > 0 else 0
        
        print(f"\n📈 Average CLIP Scores ({len(pretrained_scores)} samples):")
        print(f"  Pretrained: {avg_pretrained:.2f}/100")
        print(f"  Fine-tuned: {avg_finetuned:.2f}/100")
        print(f"  Average Improvement: {avg_improvement:+.2f} ({avg_improvement_pct:+.1f}%)")
        
        if avg_improvement > 0:
            print(f"\n✅ Fine-tuned model shows BETTER text-image alignment!")
        elif avg_improvement < 0:
            print(f"\n⚠️  Fine-tuned model shows LOWER text-image alignment")
        else:
            print(f"\n➖ No significant difference in alignment")
        
        # Visualize comparison
        fig, axes = plt.subplots(min(2, len(test_prompts)), 2, figsize=(12, 5*min(2, len(test_prompts))))
        if min(2, len(test_prompts)) == 1:
            axes = axes.reshape(1, -1)
        
        for idx, prompt in enumerate(test_prompts[:2]):
            # Generate images
            with torch.no_grad():
                gen_pretrained = pipe_pretrained(
                    prompt=prompt, num_inference_steps=50, guidance_scale=7.5,
                    generator=torch.Generator(device=device).manual_seed(42)
                ).images[0]
                
                gen_finetuned = pipe(
                    prompt=prompt, num_inference_steps=50, guidance_scale=7.5,
                    generator=torch.Generator(device=device).manual_seed(42)
                ).images[0]
            
            # Plot
            axes[idx, 0].imshow(gen_pretrained)
            axes[idx, 0].set_title(f"Pretrained\n({pretrained_scores[idx]:.1f}/100)", fontsize=10, color='blue')
            axes[idx, 0].axis('off')
            
            axes[idx, 1].imshow(gen_finetuned)
            axes[idx, 1].set_title(f"Fine-tuned\n({finetuned_scores[idx]:.1f}/100)", fontsize=10, color='green')
            axes[idx, 1].axis('off')
        
        plt.suptitle('CLIP Alignment Comparison: Pretrained vs Fine-tuned', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.savefig("../results/clip_alignment_comparison.png", dpi=100, bbox_inches='tight')
        plt.show()
        print(f"\n✓ Comparison visualization saved to: ../results/clip_alignment_comparison.png")

except Exception as e:
    print(f"⚠️  CLIP evaluation skipped: {e}")
    import traceback
    traceback.print_exc()

print("="*70 + "\n")

In [ ]:
"""
To use LoRA fine-tuning, install: pip install peft

Then replace the model loading with:

from peft import get_peft_model, LoraConfig, TaskType

# LoRA config
lora_config = LoraConfig(
    r=16,  # Rank
    lora_alpha=32,
    target_modules=["to_q", "to_v"],  # Target modules in attention layers
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.FEATURE_EXTRACTION
)

# Apply LoRA
unet = get_peft_model(unet, lora_config)
unet.print_trainable_parameters()

# Now only LoRA parameters will be trained (~1% of model)
# This significantly reduces memory and training time
"""

print(__doc__)

## 13. Summary and Next Steps

### What This Notebook Does:
✅ **Fine-tunes Stable Diffusion** on your COCO + Flickr dataset  
✅ **Trains UNet** for custom image generation  
✅ **Optionally fine-tunes text encoder** for better text understanding  
✅ **Uses pre-encoded latents** for efficient training  
✅ **Implements mixed precision** for reduced memory usage  
✅ **Saves checkpoints** for resuming training  
✅ **Generates test images** to verify quality  

### Key Features:
- **Memory Efficient**: Pre-encoding images, gradient accumulation, mixed precision
- **Flexible**: Fine-tune UNet only or + text encoder
- **Scalable**: Works on single GPU → multi-GPU → HiperGator
- **Lightweight**: LoRA option available (~80% memory reduction)

### Hyperparameters to Adjust:
- `train_batch_size`: Increase if GPU has memory (8-16 for A100)
- `learning_rate`: 1e-4 is good starting point
- `num_epochs`: 10-20 typically sufficient for fine-tuning
- `image_size`: 512x512 standard, can use 768x768 on A100
- `num_inference_steps`: 30-50 for inference quality

### Next Steps:
1. Start with small batch size and few epochs to test
2. Monitor loss curve for overfitting
3. Save best checkpoint (lowest loss)
4. Evaluate with CLIP scores or manual inspection
5. Deploy to HiperGator for larger-scale training
6. Consider LoRA for parameter-efficient fine-tuning

### Troubleshooting:
- **Out of memory**: Reduce batch size or enable gradient checkpointing
- **Slow training**: Check if data loading is bottleneck
- **Loss not decreasing**: Check learning rate, data quality
- **Bad quality images**: Train longer, increase inference steps

Good luck! 🚀